Load experience and train the behavioral cloning agent

In [ ]:
from rl_coach.agents.bc_agent import BCAgentParameters
from rl_coach.base_parameters import VisualizationParameters, TaskParameters
from rl_coach.core_types import TrainingSteps, EnvironmentSteps
from rl_coach.environments.gym_environment import Atari
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import SimpleSchedule


agent_params = BCAgentParameters()
agent_params.memory.load_memory_from_file_path = 'experience_breakout.p'
agent_params.algorithm.num_consecutive_playing_steps = EnvironmentSteps(0)
env_params = Atari(level='BreakoutDeterministic-v4')

# Create graph manager
graph_manager = BasicRLGraphManager(agent_params=agent_params, env_params=env_params,
                                              schedule_params=SimpleSchedule(), vis_params=VisualizationParameters())

# Set the checkpoint save directory
task_parameters = TaskParameters(framework_type="tensorflow",
                                         save_checkpoint_secs=10)
task_parameters.__dict__['save_checkpoint_dir'] = './checkpoints'
graph_manager.create_graph(task_parameters)

graph_manager.train_and_act(TrainingSteps(3000))

Evaluate the trained agent

In [ ]:
graph_manager.evaluate(EnvironmentSteps(1000), keep_networks_in_sync=True)
graph_manager.save_checkpoint()
#print(graph_manager.top_level_manager.agents['agent'].networks['main'])

Load trained model into a policy gradients agent and train

In [ ]:
from rl_coach.agents.policy_gradients_agent import PolicyGradientsAgentParameters
from rl_coach.filters.filter import OutputFilter
from rl_coach.environments.gym_environment import AtariInputFilter
import tensorflow as tf


# Workaround to allow running a second evaluation with the same graph manager
tf.reset_default_graph()
graph_manager.graph_creation_time = None

agent_params = PolicyGradientsAgentParameters()
agent_params.algorithm.discount = 0.99
agent_params.algorithm.apply_gradients_every_x_episodes = 5
agent_params.algorithm.num_steps_between_gradient_updates = 20000

agent_params.network_wrappers['main'].optimizer_type = 'Adam'
agent_params.network_wrappers['main'].learning_rate = 0.0005

agent_params.input_filter = AtariInputFilter()
agent_params.output_filter = OutputFilter()

graph_manager.agent_params = agent_params

# Set the checkpoint restore directory
task_parameters.__dict__['checkpoint_restore_dir'] = './checkpoints'
task_parameters.__dict__['save_checkpoint_secs'] = 600
graph_manager.create_graph(task_parameters)

graph_manager.restore_checkpoint()
graph_manager.train_and_act(TrainingSteps(100000))

Evaluate the trained agent

In [ ]:
graph_manager.evaluate(EnvironmentSteps(1000), keep_networks_in_sync=True)